In [ ]:
import scanpy as sc 
import anndata as ad
import polars as pl
import os
import sys
# Add the module_folder to the sys.path list
sys.path.append('/home/jovyan/share/data/analyses/benjamin/Single_cell_project_rapids/analysis_functions/')
from plotting import *

In [ ]:
PROJECT_DIR = "/home/jovyan/share/data/analyses/benjamin/Single_cell_project_rapids/Beactica"

In [ ]:
filtered_anndata = ad.read("/home/jovyan/share/data/analyses/benjamin/cellxgene/Beactica/deepprofiler/sc_embedding_scanpy_Beactica_deep+cell_edist_segmfix3.h5ad")

In [ ]:
filtered_anndata.obs['moa_conc'] = filtered_anndata.obs['moa'].astype(str) + '_' + filtered_anndata.obs['Metadata_cmpdConc'].astype(str)

## Compound densities

In [ ]:
compounds = ["NARCICLASINE", "BORTEZOMIB", "STAUROSPORINE", "DACTINOMYCIN (ACTINOMYCIN D)", "CISPLATIN"]
moas = list(filtered_anndata.obs["moa"].unique())

In [ ]:
adata.uns["umap_density_Metadata_cmpdNameConc_param"]

In [ ]:
for comp in top_comp:
    adata = filtered_anndata[filtered_anndata.obs["Metadata_cmpdName"].isin(['DIMETHYL SULFOXIDE', comp])]
    sc.tl.embedding_density(adata, basis='umap', groupby='Metadata_cmpdNameConc')
    sc.pl.embedding_density(adata, bg_dotsize = 40, fg_dotsize= 150, basis='umap', key='umap_density_Metadata_cmpdNameConc', save = f"figures/{comp}_density_umap_conc_sign.png")

## E distance

In [ ]:
from scperturb import *
import pandas as pd

In [ ]:
filtered_anndata = ad.read("/home/jovyan/share/data/analyses/benjamin/cellxgene/Beactica/deepprofiler/sc_embedding_scanpy_Beactica_deep+cell_edist_segmfix3.h5ad")

In [ ]:
estats = edist(filtered_anndata, obs_key='Metadata_cmpdName', obsm_key='X_pca', dist='sqeuclidean', n_jobs= -1)


In [ ]:
from scipy.stats import zscore
from scipy.cluster.hierarchy import distance, linkage, dendrogram
from scipy.cluster import hierarchy

def cluster_matrix(matrix, how='row', return_order=False, method='centroid'):
    '''
    Hierarchical clustering of a matrix/dataframe. `how` can be 'col', 'row' or 'both' (default: 'row').
    '''
    if how not in ['col', 'row', 'both']:
        raise ValueError('Value for "how" must be row or col.')
    if how!='both':
        M = matrix if how=='row' else matrix.T
        dist = distance.pdist(M)
        link = linkage(dist, method=method)
        dend = dendrogram(link, no_plot=True)
        order = np.array(dend['leaves'], dtype=int)
        if return_order:
            return order
        elif isinstance(matrix, pd.DataFrame):
            return matrix.iloc[order] if how=='row' else matrix.iloc[:, order]
        else:
            return matrix[order] if how=='row' else matrix[:, order]
    else:
        if return_order:
            warn('Returning order when clustering both row and col is not supported.')
        matrix_ = cluster_matrix(matrix, how='row', return_order=False, method=method)
        return cluster_matrix(matrix_, how='col', return_order=False, method=method)
    

In [ ]:
ed=estats

with sns.axes_style('whitegrid'):
    fig, ax  = plt.subplots(1, figsize=[20,20], dpi = 300)

sns.heatmap(cluster_matrix(ed, "both"), robust=True, cmap = sns.cm.mako_r, xticklabels=True, yticklabels=True, ax=ax)
ax.set_title('E-distance between Beactica perturbations')
ax.set_xlabel('E-distance')
ax.set_ylabel('E-distance')
plt.savefig("figures/e_distance/e_dist_heatmap_grit_filtered.png", dpi = 300)


In [ ]:
with sns.axes_style('whitegrid'):
    fig, ax  = plt.subplots(1, figsize=[20,20], dpi = 300)

Z = hierarchy.linkage(ed, 'single')
dn = hierarchy.dendrogram(Z, labels=ed.columns, color_threshold=800, ax=ax)
plt.xticks(rotation=90)
plt.grid(axis='y')
plt.ylabel('E-distance')
plt.xlabel('cell type')
plt.title('Perturbations hierarchy based on e dist')
plt.savefig("figures/e_distance/e_dist_dendogram_grit_filter_segmcorr.png", dpi = 300)
plt.show()

## E test

In [ ]:
etest_grit_all = pd.read_csv("etest_res_grit_filt_200_samples_100000_perms.csv")

In [ ]:
estats_control = pd.DataFrame(estats.loc[:, 'DIMETHYL SULFOXIDE'])
estats_control.columns=['E-statistic']
estats_control['tmp'] = np.log10(np.clip(estats_control['E-statistic'], 0, np.infty)+1)
estats_control.reset_index(inplace=True)

In [ ]:
estats_control.sort_values("tmp")

In [ ]:
e_dist_sign = pd.merge(estats_control, etest_grit_all, left_on = "Metadata_cmpdName", right_on = "Unnamed: 0", how = "left")

In [ ]:
def e_dist_violin_sign(estats, condition, sign = False):
    estats = estats[estats["Metadata_cmpdName"] != "DIMETHYL SULFOXIDE"]
    scale=0.75
    with sns.axes_style('whitegrid'):
        fig, ax = plt.subplots(figsize=[20*scale, 5*scale], dpi=300)
    # np.log10(estats_control['E-statistic']+1)
    sns.violinplot(data=estats, x='tmp', inner=None, color=".8", width=0.8, bw=0.5)
    if sign:
        ax = sns.swarmplot(data=estats, x=estats['tmp'], y=['']*len(estats), size=10, marker='$\mathbf{\odot}$',
                    edgecolors='white', linewidth=0, hue = "significant_adj",palette=['tab:blue', 'tab:red', 'tab::orange'])
        
    else:
        ax = sns.swarmplot(data=estats, x=estats['tmp'], y=['']*len(estats), size=10, marker='$\mathbf{\odot}$',
                    edgecolors='white', linewidth=0, palette=['tab:blue', 'tab:red'])
    plt.xlabel('E-distance+1 to unperturbed (log scale)')
    ax.axvline(0, c='grey', linestyle='--', linewidth=4)
    # log scale x ticks
    from matplotlib import ticker as mticker
    ax.xaxis.set_major_formatter(mticker.StrMethodFormatter("$10^{{{x:.0f}}}$"))
    xmin, xmax = ax.get_xlim()
    tick_range = np.arange(0, xmax)
    ax.xaxis.set_ticks(tick_range)
    ax.xaxis.set_ticks([np.log10(x+1) for p in tick_range for x in np.linspace(10 ** p, 10 ** (p + 1), 10)], minor=True)
    ax.set_xlim([-1, 5.5])
    if sign:
        plt.savefig(f'e_dist_violin_{condition}_sign_lev_nodead.png', bbox_inches='tight')
    else:
        plt.savefig(f'e_dist_violin_{condition}.png', bbox_inches='tight')
    plt.show()

In [ ]:
e_dist_violin_sign(e_dist_sign, "full_grit", sign = True)

In [ ]:
e_dist_sign[e_dist_sign["significant_adj"] == "False"]

In [ ]:
def show_topn_comps(edist, estats, n, control):
    etsats_sig = estats[estats["significant_adj"] == "True"]
    df_sorted = etsats_sig.sort_values(by="E-statistic", ascending=False)
    # Get top n values from target_column
    top = df_sorted.head(n)["Metadata_cmpdName"].tolist()
    # Get bottom n values from target_column
    bot= df_sorted.tail(n)["Metadata_cmpdName"].tolist()
    scale=0.3
    plt.subplots(figsize=[13*scale,10*scale], dpi=300)
    conds = ["DIMETHYL SULFOXIDE"] + bot + top
    sub = edist.loc[conds, conds] + 1
    sub = np.log10(np.clip(sub, 0, np.infty)+1)
    sns.heatmap(sub, robust=False, linewidth=3)
    plt.xticks(rotation=45, ha='right')
    #plt.savefig(f'figures/e_distance/top_bot_{n}_cmpds_heatmap.png', bbox_inches='tight')
    plt.show()
    return top, bot

In [ ]:
top_comp, bottom_comp = show_topn_comps(estats, e_dist_sign, 5, "DIMETHYL SULFOXIDE")

In [ ]:
def run_edist_top_n(adata, cmpd_list, group):
    filt_dat = adata[adata.obs["Metadata_cmpdName"].isin(cmpd_list)]
    e_stats = edist(filt_dat, obs_key=group, obsm_key='X_pca', dist='sqeuclidean', n_jobs= -1)
    
    return e_stats

In [ ]:
top5_edist = run_edist_top_n(filtered_anndata, top_comp + ["DIMETHYL SULFOXIDE"], "Metadata_cmpdNameConc")

In [ ]:
top5_edist

In [ ]:
e_dist_violin(top5_edist, ctrl = "DIMETHYL SULFOXIDE_0.1", condition = None)

In [ ]:
with sns.axes_style('whitegrid'):
    fig, ax  = plt.subplots(1, figsize=[20,20], dpi = 300)

sns.heatmap(cluster_matrix(top5_edist, "both"), robust=True, xticklabels=True, yticklabels=True, ax=ax)
ax.set_title('E-distance between Beactica perturbations')
ax.set_xlabel('E-distance')
ax.set_ylabel('E-distance')
#plt.savefig("figures/e_distance/e_dist_heatmap_top5_conc.png", dpi = 300)


In [ ]:
def edist_dose_response_curve(dist):
    reference_row = dist['DIMETHYL SULFOXIDE_0.1']
    reference_pd = pd.DataFrame(reference_row)
    reference_pd = reference_pd.reset_index()
    split_columns = reference_pd['Metadata_cmpdNameConc'].str.split('_', expand=True)

    # Assign the split columns back to the original DataFrame
    reference_pd['Treatment'] = split_columns[0]
    reference_pd['Concentration'] = split_columns[1]
    reference_pd = reference_pd[reference_pd["Treatment"] != "DIMETHYL SULFOXIDE"]
    # Convert 'Concentration' column to numeric
    reference_pd['Concentration'] = pd.to_numeric(reference_pd['Concentration'])

    data_for_plotting = reference_pd.rename(columns = {"DIMETHYL SULFOXIDE_0.1" : "E-Distance"})
    # Step 3: Create a New DataFrame
    # Step 4: Plot
    plt.figure(figsize=(15, 10), dpi = 300)
    sns.lineplot(data=data_for_plotting, x='Concentration', y='E-Distance', hue='Treatment', marker='o')
    plt.title('E distance to DMSO')
    plt.xlabel('Concentration')
    plt.ylabel('E distance')
    plt.legend(title='Treatment', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    #plt.savefig("figures/e_distance/top5_edist_dose_resp.png")
    plt.show()

In [ ]:
edist_dose_response_curve(top5_edist)

## Embedding only significant compounds

In [ ]:
filtered_anndata.obs = filtered_anndata.obs.merge(e_dist_sign, left_on="Metadata_cmpdName", right_on='Metadata_cmpdName', how='left')

In [ ]:
filtered_anndata.obs['significant'] = filtered_anndata.obs['significant'].fillna(False)

In [ ]:
filtered_anndata.obs["significant"].unique()

In [ ]:
filtered_anndata.write_h5ad("sc_embedding_scanpy_Beactica_deep+cell_edist.h5ad")